## 1. C# Language Idioms
The C# language has evolved since its first commercial release in 2001. It’s now much farther removed from C++ or Java than it was in its original release.

### Item 1: Prefer Implicitly Typed Local Variables
I prefer var and use it to declare many local variables because I find that it focuses the developer’s attention on the important part (the semantic meaning) and not on the particulars of a variable’s type. The compiler still warns me if I have created any construct that doesn’t type-check.

In [6]:
class MyClass {}

MyClass obj1 = new MyClass();
var obj2 = new MyClass();

In [10]:
Console.WriteLine(obj1.GetType().Name);

MyClass


In [9]:
Console.WriteLine(obj2.GetType().Name);

MyClass


Sometimes using var and not knowing the return type of some function can make things confusing when reading the code.

In [12]:
class GuessWho {
    public static double GetMagicNumber() {
        return 1.0;
    }
}

var num = GuessWho.GetMagicNumber(); // hmmm, int, float, double???
Console.WriteLine(num);
Console.WriteLine(num.GetType().Name);

1
Double


At other times you can shoot yourself in the foot by being type explicit in variable declarations.

For example, [IQueryable<T>](https://docs.microsoft.com/en-us/dotnet/api/system.linq.iqueryable?view=net-5.0) derives from [IEnumerable<T>](https://docs.microsoft.com/en-us/dotnet/api/system.collections.ienumerable?view=net-5.0). In the below example the second query does not call Queryable.Where; instead, it calls Enumerable.Where. That has a large negative implication for performance.

In [46]:
using System;
using System.Collections.Generic;


class Customer {
    public readonly String ContactName;
    
    public Customer(String ContactName) {
        this.ContactName = ContactName;
    } 
}


class DB {
    public IQueryable<Customer> Customers;
    
    public DB() {
        List<Customer> list = new List<Customer>();
        list.Add(new Customer("Bob"));
        list.Add(new Customer("Jane"));
        list.Add(new Customer("Jim"));        
        this.Customers = list.AsQueryable();
    }
}


class BadlyTypedQuery {
    private DB db;
    
    public BadlyTypedQuery() {
        this.db = new DB();
    }

    public IEnumerable<string> FindCustomersStartingWith1(String start) {
        IEnumerable<string> q = 
            from c in db.Customers
            select c.ContactName;
        
        var q2 = q.Where(s => s.StartsWith(start));
        return q2;
    }
    
    // Same method, but different return type.
    public IQueryable<string> FindCustomersStartingWith2(String start) {
        var q = from c in db.Customers select c.ContactName;
        var q2 = q.Where(s => s.StartsWith(start));
        return q2;
    }
}


var query = new BadlyTypedQuery();

var result1 = query.FindCustomersStartingWith1("J");
foreach(String item in result1) {
    Console.WriteLine(item);
}
Console.WriteLine(result1.GetType().Name);
// Jane, Jim
// WhereEnumerableIterator`1


var result2 = query.FindCustomersStartingWith2("J");
foreach(String item in result2) {
    Console.WriteLine(item);
}
Console.WriteLine(result2.GetType().Name);
// Jane, Jim
// EnumerableQuery`1

Jane
Jim
WhereEnumerableIterator`1
Jane
Jim
EnumerableQuery`1


You, the developer, must decide whether letting the compiler silently declare the compile-time type of the variable harms readability. If not being able to immediately see the exact type of a local variable creates ambiguity when someone reads the code, it’s best to declare that type explicitly. However, in many cases, the code clearly conveys the semantic information about the variable.

### Item 2: Prefer readonly to const
You declare runtime constants with the readonly keyword. Compile-time constants are declared with the const keyword. Compile-time constants are slightly faster, but far less flexible.

In [49]:
// Complile-time constant
const int Millennium = 2000;

// Runtime constant
static readonly int ThisYear = 2021;

Console.WriteLine(Millennium);
Console.WriteLine(ThisYear);

2000
2021


In [50]:
Console.WriteLine(Millennium.GetType().Name);

Int32


In [51]:
Console.WriteLine(ThisYear.GetType().Name);

Int32


A compile-time constant is replaced with the value of that constant in your object code. This distinction places several restrictions on when you are allowed to use either type of constant. Compile-time constants can be used only for the built-in integral and floating-point types, enums, or strings.

Read-only values are also constants in that they cannot be modified after the constructor has executed. But read-only values are different in that they are assigned at runtime. You have much more flexibility when working with runtime constants. For one thing, runtime constants can be any type.

In [52]:
class ReadOnlyExample {
    public readonly int num;
    
    public ReadOnlyExample() {
        this.num= 7;
    }
}

var roe = new ReadOnlyExample();

In [54]:
roe.num

7

In [57]:
roe.num = 8; // not after the constructor has run

Unhandled exception: (1,1): error CS0191: A readonly field cannot be assigned to (except in a constructor or a variable initializer)

### Item 3: Prefer the is or as Operators to Casts
The correct choice is to use the as operator whenever you can because it is safer than blindly casting and is more efficient at runtime.

In [61]:
// one way to do it
class MyType {}
class NotMyType {}

class Factory {
    public static MyType GetObject() {
        return new MyType();
    }
}

object obj = Factory.GetObject();
MyType mt = obj as MyType;
if (mt != null) {
    Console.WriteLine("Success casting " + mt.GetType().Name);
} else {
    Console.WriteLine("No luck in casting obj!");
}

Success casting MyType


In [63]:
object obj = new NotMyType();
MyType mt = obj as MyType;
if (mt != null) {
    Console.WriteLine("Success casting " + mt.GetType().Name);
} else {
    Console.WriteLine("No luck in casting obj!");
}

No luck in casting obj!


In [64]:
// another way to cast
object obj = Factory.GetObject();
try {
    MyType mt;
    mt = (MyType)obj;
    Console.WriteLine("Success casting " + mt.GetType().Name);
} catch (InvalidCastException) {
    Console.WriteLine("No luck in casting obj!");
}

Success casting MyType


You’ll agree that the first version is simpler and easier to read. It does not have the try/catch clause, so you avoid both the overhead and the code. Notice that the cast version must check null in addition to catching exceptions.

as and is operators examine the runtime type of the object being converted; they do not perform any other operations, except boxing when necessary. If a particular object is not the requested type or is derived from the requested type, they fail. Casts, on the other hand, can use conversion operators to convert an object to the requested type. This includes any built-in numeric conversions. Casting a long to a short can lose information.

In [73]:
// version one
class SecondType {
    private MyType _value;
    
    public static implicit operator MyType(SecondType t) {
        return t._value;
    }
}


class Factory {
    public static SecondType GetObject() {
        return new SecondType();
    }
}


object obj = Factory.GetObject(); // obj is a SecondType
MyType mt = obj as MyType; // Fails
if (mt != null) {
    Console.WriteLine("Conversion success...");
} else {
    Console.WriteLine("Conversion failed!");
}

Conversion failed!


In [75]:
// version two
try {
    MyType mt;
    mt = (MyType)obj; // Fails
    Console.WriteLine("Conversion success...");
} catch (InvalidCastException) {
    Console.WriteLine("Conversion failed!");
}

Conversion failed!


Both versions fail. It fails because your compiler is generating code based on the compile-time type of the object, o. The compiler knows nothing about the runtime type of o; it views o as an instance of object.

The foreach statement needs to use casts to support both value types and reference types. By choosing the cast operator, the foreach statement exhibits the same behavior, no matter what the destination type is. However, because a cast is used, foreach loops can cause an InvalidCastException to be thrown.

In [91]:
List<object> objects = new List<object>();
objects.Add(1);
objects.Add(2);
objects.Add(3);
objects.Add("4");

foreach(int i in objects) {
    Console.WriteLine(i.ToString() + " " + i.GetType().Name);
}

1 Int32
2 Int32
3 Int32


Unhandled exception: System.InvalidCastException: Unable to cast object of type 'System.String' to type 'System.Int32'.
   at Submission#94.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location where exception was thrown ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [97]:
// to get around this...
foreach(object obj in objects) {
    try {
        int i;
        i = (int)obj;
        Console.WriteLine(i + " " + i.GetType().Name);
    } catch (InvalidCastException) {
        Console.WriteLine("Unable to cast " + obj.GetType().Name + " to int -> " + obj);
    }
}

1 Int32
2 Int32
3 Int32
Unable to cast String to int -> 4


### Item 4: Replace string.Format() with Interpolated Strings